In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
%matplotlib inline
import sys
from sklearn.linear_model import LinearRegression 
from sklearn import metrics
from sklearn import preprocessing
import statsmodels.api as sm

In [ ]:
train=pd.read_csv('/kaggle/input/av-healthcare-analytics-ii/healthcare/train_data.csv')
test=pd.read_csv('/kaggle/input/av-healthcare-analytics-ii/healthcare/test_data.csv')
sub=pd.read_csv('/kaggle/input/av-healthcare-analytics-ii/healthcare/sample_sub.csv')

# Data Exploration

In [ ]:
train.head()

In [ ]:
train.info()


In [ ]:
train.isna().sum()

## we checked for missing values, as we can see most columns of the dataset don't have missing values.
  ## this shows that the data is pre-processed and cleaned.

In [ ]:
train.head()

In [ ]:
train['count']=1

In [ ]:
train.head()

# Data Visualization

In [ ]:
train.Stay.value_counts()

### this shows that LOS that ranges between 21-30 days is the most frequent LOS
### LOS with 11-20 days is the 2nd most freq LOS

In [ ]:
train.Age.value_counts()

### Age groups 41-50 and 31-40 are the most freq group hospitalized

In [ ]:
train['Severity of Illness'].describe()

### the most frequent severity of illness is "Moderate" severity

In [ ]:
train['Hospital_type_code'].describe()

### there are 7 unique hospital type codes with 'a' being the most frequent code which shows that the density of patient cases will be highest in hospital type code 'a'

# Bivariate Analysis

In [ ]:
plt.figure(figsize = (10,5))
sns.countplot(x = 'Hospital_type_code', data = train, palette = 'gist_earth')
plt.xlabel('Hospital type code', size = 20)
plt.ylabel('Patient Density', size = 20)
plt.title('Patient Density per Hospital Type Code')
plt.show()

### this confirms what I stated before : hospital with type code'a' has highest number of patients
### thus, less beds/rooms left available. hospital with type code'g' has lowest number pf patients
### thus, more beds/rooms left available. 
### But what is the LOS of most patients in hospital with type code'a'?
### what is the LOS of most patients in hospital with type code'g'?
### LOS will determine how often the beds become readily available regardless of current 
### patient density.

In [ ]:
plt.figure(figsize = (10,5))
sns.countplot(x = 'Hospital_type_code', data = train, palette = 'rainbow')
plt.xlabel('Hospital type code', size = 20)
plt.ylabel('Stay', size = 20)
plt.title('Length of Stay per Hospital Type Code')
plt.show()

In [ ]:
train.Stay.value_counts()

In [ ]:
Stay_index = train.Stay.value_counts().index[:11]
Stay_index

In [ ]:
Hospital_type_codeindex= train.Hospital_type_code.value_counts().index[:7]
Hospital_type_codeindex

In [ ]:
subdata = train[(train.Hospital_type_code.isin(Hospital_type_codeindex)) & (train.Stay.isin(Stay_index))]

In [ ]:
cf = pd.crosstab(columns=subdata.Stay, index = subdata.Hospital_type_code) 
cf

In [ ]:
plt.figure()
cf = pd.crosstab(columns=subdata.Stay, index = subdata.Hospital_type_code) 
cf.plot.bar(figsize = (20,8), )
plt.legend(loc = 'best')
plt.show()


### Most patients in all hopital type codes had a LOS= 21-30 followed by 11-20

# Data Wrangling

In [ ]:
train.Hospital_region_code.describe()

In [ ]:
train.Hospital_region_code.value_counts()

In [ ]:
import plotly.express as px
fig = px.sunburst(train, path=['Hospital_region_code', 'Hospital_type_code'])
fig.update_layout(title='Categorizing Hospital Type according to Region')
fig.show()

### This shows that patient density is very close within X and Y region. in the X region the patients are better distributed between different hospital codes than regions Y and Z.
 

### These indep variables are not valuable for our task. The aim of this project is to predict the LOS. Predicting LOS helps hospitals to identify patients of high LOS risk (patients who will stay longer) at the time of admission and thus better allocate resources for the management of these patients for the purposse of decreasing that long LOS.Thus, how patients are distributed betweeen different hospital type codes and region is not important here at all. What we care about is how to better manage patient at risk of high LOS in each hospital.

### For this matter I have decided to drop:case_id,Hospital_code,City_Code_Hospital,Hospital_region_code,City_Code_Patient,Visitors with patient,Admission_Deposit
### I will keep the indep variable Hospital_type_code

In [ ]:
train

In [ ]:
train.Ward_Type.value_counts()

In [ ]:
train.Ward_Facility_Code.value_counts()

### I will drop Ward_Type and Ward_Facility_Code as well. As they don't play any role in predicting our dep variable LOS.

In [ ]:
train_new=train.drop(['case_id','Hospital_code','City_Code_Hospital','Hospital_region_code','City_Code_Patient','Visitors with Patient','Admission_Deposit','Ward_Facility_Code','Ward_Type'],axis=1)

In [ ]:
train_new

### the patientid in the first 5 observations is the same: 31397. Is this re-admission ? 31397 mentioned twice in hospital type code c and twice in b.
### as a healthcare professional I know that the patient is given same MRN regardless of number of encounters. It is alarming because the LOS ASSOCIATED with that patientid is very high 41-50 days. Maybe there are duplicated rows? rows 3 and 4 seem duplicated. Let me drop one of them.

In [ ]:
train_new.drop(train_new.index[4])

### The questions we need to answer to predict patients at risk of high LOS:
### - What department has the greater density of patients with highest LOS?
### - Can bed grade affect LOS? Can it play a role in decreasing or increasing LOS?
### - What type of Admission is associated with highest LOS?
### - What level of severity of illness is associated with highest LOS?
### - What Age group is associated with highest LOS?

### We will answer these questions in the following visualizations.


In [ ]:
train_new=train_new.drop(['Hospital_type_code','patientid'],axis=1)

In [ ]:
train_new

### Available extra rooms in hospital 

In [ ]:
plt.figure(figsize=(12, 6))
sns.countplot(train_new['Available Extra Rooms in Hospital']) 

### Patient denisty per department

In [ ]:
plt.figure(figsize=(12, 6))
sns.countplot(train.Department) 


### This doesn't tell me much. Highest patient density (patient admissions) is in Gyn department. Typically for the LOS won't go over 2-3 days. Unless there's complications with mom or baby. Many departments are not included in this dataset; especially departments that tend to have patients with high LOS.
### Where's the ICU? DOU? MedSurg? Telemetry? NICU? 

### Since the significant and busiest units of the hospital, where the MAJORITY of patients stay for care, are missing. I conclude that this dataset is incomplete. One can't possibly build a model to predict LOS using a dataset in which the vast majority of its records is from the labor & delivery unit. Any predictive ML model built using this dataset would do poorly and would give a misrepresentation of the predicted LOS. 